In [2]:
import STTModule as stt
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import timer
import time
import counter
import konlpy
import TextToSpeech as tts
from konlpy.tag import *
import collections
okt = Okt()
#!pip install twilio

from twilio.rest import Client
from selenium import webdriver
import datetime
import vlc
real_time=1
# Your Account Sid and Auth Token from twilio.com/console
# DANGER! This is insecure. See http://twil.io/secure
account_sid = "ACebbe7f72649d9547208438d6ed522a0f"
auth_token = "6b9e5e3030853565d2bb70830b089269"
mynumber="+19138283330"
client = Client(account_sid, auth_token)

bucket_name="speech_to_text_vol1"
source_file_name="output.wav"
destination_blob_name="out.wav"
real_time=1

# 1. 웹 실행( 유저 정보 저장)

In [3]:
from selenium import webdriver
driver=webdriver.Chrome(executable_path="../chromedriver.exe")
url='C:/Users/franc_000/Desktop/다빈치SW/final.html'
driver.get(url)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\franc_000\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-5834beac3209>", line 2, in <module>
    driver=webdriver.Chrome(executable_path="../chromedriver.exe")
  File "C:\Users\franc_000\Anaconda3\lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 81, in __init__
    desired_capabilities=desired_capabilities)
  File "C:\Users\franc_000\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 157, in __init__
    self.start_session(capabilities, browser_profile)
  File "C:\Users\franc_000\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 252, in start_session
    response = self.execute(Command.NEW_SESSION, parameters)
  File "C:\Users\franc_000\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_resp

SessionNotCreatedException: Message: session not created
from chrome not reachable
  (Session info: chrome=78.0.3904.97)


In [46]:
og_data=driver.find_element_by_xpath('//*[@id="list"]').text
data_list=og_data.split('\n')
import pandas as pd
phone_data=pd.DataFrame([['phone1',data_list[0]],['phone2',data_list[1]],['phone3',data_list[2]]],columns=['index','number'])
phone_data.to_csv('phone.csv',index=False)
times=pd.DataFrame([[data_list[3],data_list[4]],[data_list[7],data_list[6]]],columns=['hour','minute'])
times.to_csv('timer.csv',index=False)
driver.close()

IndexError: list index out of range

# 2. AI 실행

In [ ]:
instance = vlc.Instance()
player = instance.media_player_new()
media = instance.media_new('start.mp3')
player.set_media(media)
player.play()
time.sleep(3)

while True:
    user_data=pd.read_csv('true.csv',encoding='utf-8')
    fitted_val_list=[]
    real_time = 1
    sec=3
    null_speech = 0 #무응답
    timer_null_speech = 0 # 점심/저녁 시간에 무응답 인덱스
    index=1
    
    ########################
    #####한 달 지난 후######
    ########################
    if (len(user_data)>=30):
        while (real_time <=86400/sec):
            #print("AAAAAA"+str(time)+"AAAAA")
            bucket_name="speech_to_text_vol1"
            source_file_name="output.wav"
            destination_blob_name="out.wav"

            #녹음실행
            
            stt.record(sec)
            #저장된 .wav 파일을 Gcloud 상에 업로드
            stt.upload_blob(bucket_name, source_file_name, destination_blob_name)
            #업로드 된 파일을 불러와서 SST실시 후 저장
            response = stt.transcribe_gcs("gs://speech_to_text_vol1/out.wav")
            with open("result.txt", "w") as script:
                for result in response.results:
                    script.write(u'{}'.format(result.alternatives[0].transcript)+"\n")
            print("completed")
            sentiment_data=stt.analyze('result.txt')
            sentence_sentiment=0 # 여기에 감정분석 결과값
            sentiment_avg = 0.0;
            total = 0;
            for index, sentence in enumerate(sentiment_data.sentences):
                sentence_sentiment = sentence.sentiment.score
                total = total + sentence_sentiment
            sentiment_avg=total/2#################################################
            print(sentiment_avg)
            if(sentiment_avg<= -0.2 and real_time ==2): #############################################
                message = client.messages.create(body='사용자의 지속적인 부정감정이 관측됩니다.',from_=mynumber,to='+821086247756')
            
            model = smf.ols(formula = 'user ~ google', data = user_data)
            result = model.fit()
            beta0_hat=result.params[0]
            beta1_hat=result.params[1]
            print(sentence_sentiment)
            
            
            if (type(sentence_sentiment)==int and sentence_sentiment==0):
                fitted_val=0
                null_speech +=1
            else:
                fitted_val=beta0_hat + beta1_hat*sentence_sentiment
                null_speech=0
            fitted_val_list.append(fitted_val)
            
            #print(fitted_val)
            real_time+=1
            #print(fitted_val_list)
            
            #####################
            ####무응답 발생시####
            #####################
            #if(null_speech == 7200/sec) :
                #driver=webdriver.Chrome(executable_path="../chromedriver.exe")
                #driver.get(url)
            if (null_speech == 14400/sec) :
                message = client.messages.create(body='2시간 이상 사용자의 응답이 없습니다.',from_=mynumber,to='+821086247756')
            
            
            dt = datetime.datetime.now()
            now_time=str(dt.hour)+":"+str(dt.minute)
            now_time
            times=pd.read_csv('timer.csv')
            times_str_list=[str(times.loc[0][0])+":"+str(times.loc[0][1]),str(times.loc[1][0])+":"+str(times.loc[1][1])]
            times_str_list
            if now_time in times_str_list :
                ###### 말 걸기 ######
                #####################
                instance = vlc.Instance()
                player = instance.media_player_new()
                media = instance.media_new('hello.mp3')
                player.set_media(media)
                player.play()
                time.sleep(3)


                #녹음
                stt.record(4)
                stt.upload_blob(bucket_name, source_file_name, destination_blob_name)
                #업로드 된 파일을 불러와서 SST실시 후 저장
                response = stt.transcribe_gcs("gs://speech_to_text_vol1/out.wav")
                with open("result.txt", "w") as script:
                                for result in response.results:
                                    script.write(u'{}'.format(result.alternatives[0].transcript)+"\n")
                print("completed")

                answer1=open('result.txt').readlines()[0]
                counter22=counter.counter22()
                counter33=counter.counter33()
                print(answer1)
                answer1_list=answer1.split(' ')
                nouns_list=okt.nouns(answer1)
                print(1)
                for ans in answer1_list :
                    if ans in counter33:
                        print(2)
                        instance = vlc.Instance()
                        player = instance.media_player_new()
                        media = instance.media_new('counter3.mp3')
                        player.set_media(media)
                        player.play()
                        time.sleep(3)
                        #녹음
                        stt.record(4)
                        stt.upload_blob(bucket_name, source_file_name, destination_blob_name)
                        #업로드 된 파일을 불러와서 SST실시 후 저장
                        response = stt.transcribe_gcs("gs://speech_to_text_vol1/out.wav")
                        with open("result.txt", "w") as script:
                            for result in response.results:
                                 script.write(u'{}'.format(result.alternatives[0].transcript)+"\n")
                        print("completed")
                        answer2=open('result.txt',encoding='utf-8').readlines()[0]
                        #answer2_list=answer2.split('')
                        for ans in answer2_list:
                            if ans in counter22:
                                print(4)
                                instance = vlc.Instance()
                                player = instance.media_player_new()
                                media = instance.media_new('counter2.mp3')
                                player.set_media(media)
                                player.play()
                                time.sleep(3)
                                break
                            if ans in counter11:
                                print(5)
                                instance = vlc.Instance()
                                player = instance.media_player_new()
                                media = instance.media_new('counter1.mp3')
                                player.set_media(media)
                                player.play()
                                time.sleep(3)
                                break
                            else:
                                break
                        break;
                    if ans in counter22:
                        instance = vlc.Instance()
                        player = instance.media_player_new()
                        media = instance.media_new('counter2.mp3')
                        player.set_media(media)
                        player.play()
                        time.sleep(3)
                        break;
                    else:    
                        instance = vlc.Instance()
                        player = instance.media_player_new()
                        media = instance.media_new('counter4.mp3')
                        player.set_media(media)
                        player.play()
                        time.sleep(3)
                        #녹음
                        stt.record(4)
                        stt.upload_blob(bucket_name, source_file_name, destination_blob_name)
                        #업로드 된 파일을 불러와서 SST실시 후 저장
                        response = stt.transcribe_gcs("gs://speech_to_text_vol1/out.wav")
                        with open("result.txt", "w") as script:
                            for result in response.results:
                                 script.write(u'{}'.format(result.alternatives[0].transcript)+"\n")
                        print("completed")
                        answer2=open('result.txt').readlines()[0]
                        nouns_list=nouns_list + okt.nouns(answer2) 
                        count = collections.Counter(nouns_list)

                        tts.synthesize_text("저도 그거 참 좋아하는데요. 나중에 세상이 좋아지면 "+count.most_common()[0][0]+" 저랑 꼭 같이 먹어요.")
                        instance = vlc.Instance()
                        player = instance.media_player_new()
                        media = instance.media_new('TTS_output.mp3')
                        player.set_media(media)
                        player.play()
                        time.sleep(3)
                        break
'''      
    ###################
    ####한 달 이내#####
    ###################
    else :
        while (time<24):
            #print("AAAAAA"+str(time)+"AAAAA")
            bucket_name="speech_to_text_vol1"
            source_file_name="output.wav"
            destination_blob_name="out.wav"

            #녹음실행
            sec=3
            stt.record(sec)
            #저장된 .wav 파일을 Gcloud 상에 업로드
            stt.upload_blob(bucket_name, source_file_name, destination_blob_name)
            #업로드 된 파일을 불러와서 SST실시 후 저장
            response = stt.transcribe_gcs("gs://speech_to_text_vol1/out.wav")
            with open("result.txt", "w") as script:
                for result in response.results:
                    script.write(u'{}'.format(result.alternatives[0].transcript)+"\n")
            print("completed")
            sentiment_data=stt.analyze('result.txt')
            sentence_sentiment=0 # 여기에 감정분석 결과
            sentiment_avg = 0.0;
            total = 0;
            for index, sentence in enumerate(sentiment_data.sentences):
                sentence_sentiment = sentence.sentiment.score
                total = total + sentence_sentiment
                if(index==30):
                    break   
            sentiment_avg=total/30
            print(sentiment_avg)
            if(sentiment_avg<= -0.7):
                message = client.messages.create(body='2시간 이상 사용자의 응답이 없습니다.',from_=mynumber,to='+821086247756')
               ''' 

Recording
Finished recording
File output.wav uploaded to out.wav.
completed
Sentence 0 has a sentiment score of 0.10000000149011612
Overall Sentiment: score of 0.10000000149011612 with magnitude of 0.10000000149011612
0.05000000074505806
0.10000000149011612
Recording
Finished recording
File output.wav uploaded to out.wav.
completed
Overall Sentiment: score of 0.0 with magnitude of 0.0
0.0
0
Recording
Finished recording
File output.wav uploaded to out.wav.
completed
Sentence 0 has a sentiment score of 0.0
Overall Sentiment: score of 0.0 with magnitude of 0.0
0.0
0.0
Recording
Finished recording
File output.wav uploaded to out.wav.
completed
Sentence 0 has a sentiment score of 0.0
Overall Sentiment: score of 0.0 with magnitude of 0.0
0.0
0.0
Recording
Finished recording
File output.wav uploaded to out.wav.
completed
Sentence 0 has a sentiment score of 0.5
Overall Sentiment: score of 0.5 with magnitude of 0.5
0.25
0.5
Recording
Finished recording
File output.wav uploaded to out.wav.
compl

['12:10', '18:40']